In [1]:
import pandas as pd
import numpy as np

# ==========================
# 0. 基本設定
# ==========================

TOP_K = 5  # 每個 symptom_label 保留出現次數最多的前 K 種 message
DATA_DIR = "../data"  # 你資料夾的位置，自己改

rework_path = f"{DATA_DIR}/Rework_result.csv"
prop_path   = f"{DATA_DIR}/symptom_label_propotion.csv"
manual_path = f"{DATA_DIR}/Astoria GA_GLP DebugTree.csv"  # 先放著，之後再處理說明書

rework = pd.read_csv(rework_path)

In [2]:
# ==========================
# 2. 方法 A：把「冷門的 symptom_message」合併成 OTHER
# ==========================

# (2-1) 先算出每個 (label, message) 出現幾次
msg_freq = (
    rework
    .groupby(["fail_symptom_label", "symptom_message"])
    .size()
    .reset_index(name="n_occ")
)

# (2-2) 對每個 label 內，依照 message 出現次數由大到小排序
msg_freq = msg_freq.sort_values(
    ["fail_symptom_label", "n_occ"],
    ascending=[True, False]
)

# (2-3) 在每個 label 裡做排名（1, 2, 3, ...）
msg_freq["msg_rank"] = (
    msg_freq
    .groupby("fail_symptom_label")
    .cumcount() + 1
)

# (2-4) 排名前 TOP_K 的 → 保留原本 message，其餘 → 標成 "OTHER"
msg_freq["symptom_message_group"] = np.where(
    msg_freq["msg_rank"] <= TOP_K,
    msg_freq["symptom_message"],
    "OTHER"
)

# (2-5) 把分組結果 merge 回 rework，每一列多一個欄位 symptom_message_group
rework = rework.merge(
    msg_freq[["fail_symptom_label", "symptom_message", "symptom_message_group"]],
    on=["fail_symptom_label", "symptom_message"],
    how="left"
)

In [3]:
# ==========================
# 3. 把 repair_outcome 轉成 0/1：is_success
# ==========================

# repair_outcome 是字串，不好直接算平均，所以轉成數值：
#   SUCCESS -> 1
#   其他(FAILED, UNKNOWN...) -> 0  （你也可以改成 UNKNOWN = NaN）
rework["is_success"] = (rework["repair_outcome"] == "SUCCESS").astype(int)

In [4]:
# ==========================
# 4. 以 (symptom_label, message_group) 計算成功率
# ==========================

# 透過 groupby 算：
#   n_reworks   : 這個 (label, message_group) 出現幾次
#   success_rate: 成功比例 = 平均的 is_success
sym_msg_stats = (
    rework
    .groupby(["fail_symptom_label", "symptom_message_group"])["is_success"]
    .agg(n_reworks="size", success_rate="mean")
    .reset_index()
    .rename(columns={
        "fail_symptom_label": "symptom_label",
        "symptom_message_group": "symptom_message_grp"
    })
)

In [5]:
# ==========================
# 5. 讀 symptom_label_propotion：各 test_step 比例
# ==========================

# 這個檔應該長這樣：
#   symptom_label, test_step, cnt, total_cnt, ratio
sym_prop = pd.read_csv(prop_path)

# (5-1) 算每個 symptom_label 的「主站別」（ratio 最大那一站）
primary_step = (
    sym_prop
    .sort_values(["symptom_label", "ratio"], ascending=[True, False])
    .groupby("symptom_label")
    .head(1)
    [["symptom_label", "test_step", "ratio"]]
    .rename(columns={
        "test_step": "primary_test_step",
        "ratio": "primary_ratio"
    })
)

# (5-2) 算這個 symptom_label 出現在幾個不同的 test_step
multi_step_flag = (
    sym_prop
    .groupby("symptom_label")["test_step"]
    .nunique()
    .reset_index(name="n_steps")
)
multi_step_flag["multi_step_flag"] = multi_step_flag["n_steps"] > 1

# (5-3) 把 label 等級的 metadata 合併起來
label_meta = primary_step.merge(
    multi_step_flag,
    on="symptom_label",
    how="left"
)

In [6]:
# ==========================
# 6. 把 label metadata merge 進 (label, message_group) 統計表
# ==========================

sym_msg_stats = sym_msg_stats.merge(
    label_meta,
    on="symptom_label",
    how="left"
)


In [7]:
# ==========================
# 7. （選配）產生 sym_key 方便後面 RAG / join 說明書
# ==========================

def make_key(df, label_col, msg_col):
    """把 label + message_group 組成一個 key（全部轉小寫＋trim）"""
    return (
        df[label_col].fillna("").str.strip().str.lower()
        + " || "
        + df[msg_col].fillna("").str.strip().str.lower()
    )

sym_msg_stats["sym_key"] = make_key(
    sym_msg_stats,
    "symptom_label",
    "symptom_message_grp"
)

# 如果你之後也想在 rework 明細裡用同樣的 key：
rework["sym_key"] = make_key(
    rework,
    "fail_symptom_label",
    "symptom_message_group"
)

In [8]:
import pandas as pd

manual_path = f"{DATA_DIR}/Astoria GA_GLP DebugTree.csv"
manual_raw = pd.read_csv(manual_path)
new_header = manual_raw.iloc[0]

# 內容從第 1 列開始
manual = manual_raw.iloc[1:].copy()

# 把欄名換成 new_header
manual.columns = new_header
manual = manual.reset_index(drop=True)

In [9]:
manual.columns = [
    "Symptom label",
    "Symptom message",
    "cycle1_category",
    "cycle1_suggestion",
    "cycle2_category",
    "cycle2_suggestion",
    "cycle3_category",
    "cycle3_suggestion",
    "cycle4_category",
    "cycle4_suggestion",
    "cycle5_category",
    "cycle5_suggestion",
]


## 處理完資料了 開始整合

In [10]:
import pandas as pd

def norm(s):
    return (
        s.astype(str)
         .str.strip()
         .str.lower()
    )

manual["sym_key"] = (
    norm(manual["Symptom label"]) + " || " +
    norm(manual["Symptom message"])
)


In [11]:
def build_debug_text(row):
    parts = []

    for k in range(1, 6):   # 你現在有 cycle1 ~ cycle5
        cat_col = f"cycle{k}_category"
        sug_col = f"cycle{k}_suggestion"

        cat = row.get(cat_col)
        sug = row.get(sug_col)

        if pd.notna(cat) or pd.notna(sug):
            parts.append(f"[Cycle {k}] Category: {cat}")
            parts.append(f"[Cycle {k}] Steps: {sug}")

    return "\n".join([p for p in parts if p])  # 把 None / 空字串去掉

manual["debug_text"] = manual.apply(build_debug_text, axis=1)


In [12]:
# 例：rework_result 已經有 sym_key ＋ repair_outcome
rework["is_success"] = (rework["repair_outcome"] == "SUCCESS").astype(int)

sym_stats = (
    rework
    .groupby("sym_key")["is_success"]
    .agg(n_reworks="size", success_rate="mean")
    .reset_index()
)


In [13]:
rag_df = manual.merge(sym_stats, on="sym_key", how="left")

In [14]:
rag_df["has_stats"] = rag_df["n_reworks"].notna()
rag_df["has_stats"].mean()   # 例如 0.7 代表 70% 有 match


np.float64(0.10927152317880795)

In [15]:
manual_keys = set(manual["sym_key"])
sym_keys    = set(sym_stats["sym_key"])

overlap = manual_keys & sym_keys

print("manual keys:", len(manual_keys))
print("sym_stats keys:", len(sym_keys))
print("overlap keys:", len(overlap))
print("overlap ratio:", len(overlap) / len(manual_keys))


manual keys: 302
sym_stats keys: 284
overlap keys: 33
overlap ratio: 0.10927152317880795


In [16]:
# 1) 以 label 算 rework 統計
label_stats = (
    rework
    .groupby("fail_symptom_label")["is_success"]
    .agg(n_reworks="size", success_rate="mean")
    .reset_index()
    .rename(columns={"fail_symptom_label": "symptom_label"})
)

# 2) 統一大小寫/空白
def norm_text(s):
    return str(s).strip().lower()

label_stats["label_norm"] = label_stats["symptom_label"].apply(norm_text)
manual["label_norm"]      = manual["Symptom label"].apply(norm_text)

# 3) 用 label_norm 來 merge
rag_by_label = manual.merge(
    label_stats[["label_norm", "n_reworks", "success_rate"]],
    on="label_norm",
    how="left"
)

rag_by_label["has_stats"] = rag_by_label["n_reworks"].notna()
rag_by_label["has_stats"].mean()


np.float64(0.6887417218543046)

In [17]:
import numpy as np
import pandas as pd

# 1. 先在 rework 裡建 is_success
rework["is_success"] = (rework["repair_outcome"] == "SUCCESS").astype(int)

# 2. 以 fail_symptom_label 做統計
label_stats = (
    rework
    .groupby("fail_symptom_label")["is_success"]
    .agg(n_reworks="size", success_rate="mean")
    .reset_index()
)


In [18]:
def norm_text(s):
    return str(s).strip().lower()

# manual：建標準化 label
manual["label_norm"] = manual["Symptom label"].apply(norm_text)

# label_stats：同樣處理
label_stats["label_norm"] = label_stats["fail_symptom_label"].apply(norm_text)

# merge 進 manual
rag_df = manual.merge(
    label_stats[["label_norm", "n_reworks", "success_rate"]],
    on="label_norm",
    how="left"
)

# 補 NA：沒有 rework 記錄的就當作 0 次，success_rate 留 NaN 當「未知」
rag_df["n_reworks"] = rag_df["n_reworks"].fillna(0).astype(int)
# success_rate 你可以選擇不補，保留 NaN 表示「沒有資料」


In [19]:
def build_context(row):
    parts = []

    # 基本資訊
    if pd.notna(row["Symptom label"]):
        parts.append(f"Symptom label: {row['Symptom label']}")
    if pd.notna(row["Symptom message"]):
        parts.append(f"Symptom message: {row['Symptom message']}")

    # cycle1~5 的建議（你之前 build_debug_text 做好的）
    if pd.notna(row.get("debug_text")) and str(row["debug_text"]).strip():
        parts.append(f"Debug tree steps:\n{row['debug_text']}")

    # rework 統計
    if row["n_reworks"] > 0:
        parts.append(
            f"Historical reworks for this label: {row['n_reworks']} cases, "
            f"success rate: {row['success_rate']:.2f}"
        )
    else:
        parts.append("Historical reworks for this label: no data in rework logs.")

    return " | ".join(parts)

rag_df["context"] = rag_df.apply(build_context, axis=1)


In [20]:
# 建一個 doc_id 當唯一 ID
rag_df["doc_id"] = rag_df.index.astype(str)

rag_df_sub = rag_df[[
    "doc_id",
    "Symptom label",
    "Symptom message",
    "context",
    "n_reworks",
    "success_rate",
    "label_norm"
]].rename(columns={
    "Symptom label": "symptom_label",
    "Symptom message": "symptom_message"
})


In [21]:
rag_df_sub

,doc_id,symptom_label,symptom_message,context,n_reworks,success_rate,label_norm
0,0,Diorite SSH Connection Failed,Create Diorite SSH Connection failed.\nCan not...,Symptom label: Diorite SSH Connection Failed |...,471,0.919321,diorite ssh connection failed
1,1,Checking BMC boot readiness and get BMC IP Failed,Failed to get IP address via telnet\nResponse ...,Symptom label: Checking BMC boot readiness and...,558,0.890681,checking bmc boot readiness and get bmc ip failed
2,2,Checking BMC boot readiness and get BMC IP Failed,Switch to agora console failed.,Symptom label: Checking BMC boot readiness and...,558,0.890681,checking bmc boot readiness and get bmc ip failed
3,3,DIMM infos validation failure,Component DIMM SN: (L)64GB 2RX4 PC5-4800B-RA0-...,Symptom label: DIMM infos validation failure |...,282,0.851064,dimm infos validation failure
4,4,Capture Diorite Console and Attach to log Failed,Caught Exception exception.,Symptom label: Capture Diorite Console and Att...,8,1.000000,capture diorite console and attach to log failed
...,...,...,...,...,...,...,...
297,297,pcierrors[GA],PE2,Symptom label: pcierrors[GA] | Symptom message...,0,NaN,pcierrors[ga]
298,298,pcierrors[GA],PE3,Symptom label: pcierrors[GA] | Symptom message...,0,NaN,pcierrors[ga]
299,299,pcierrors[GA],PE4,Symptom label: pcierrors[GA] | Symptom message...,0,NaN,pcierrors[ga]
300,300,pcierrors[GA],PE5,Symptom label: pcierrors[GA] | Symptom message...,0,NaN,pcierrors[ga]


## Repair資料庫

In [22]:
def retrieve_by_label(rag_df_sub, fail_symptom_label, top_k=5):
    key = str(fail_symptom_label).strip().lower()
    
    cand = rag_df_sub[rag_df_sub["label_norm"] == key].copy()
    
    # 如果同一個 label 有很多行，就按照「出現次數 & 成功率」排序
    cand = cand.sort_values(
        ["n_reworks", "success_rate"],
        ascending=[False, False]
    )
    return cand.head(top_k)


In [23]:
rows = retrieve_by_label(rag_df_sub, "Checking BMC boot readiness and get BMC IP Failed", top_k=3)
rows[["symptom_label", "n_reworks", "success_rate"]]

,symptom_label,n_reworks,success_rate
1,Checking BMC boot readiness and get BMC IP Failed,558,0.890681
2,Checking BMC boot readiness and get BMC IP Failed,558,0.890681


In [24]:
def recommend_repair_simple(fail_symptom_label, top_k=3):
    # 1. Retrieve the historical repair records for this label
    rows = retrieve_by_label(rag_df_sub, fail_symptom_label, top_k=top_k)
    
    if rows.empty:
        return None, f"No historical repair data found for symptom_label = '{fail_symptom_label}'."

    # 2. Build context block
    docs = []
    for i, (_, r) in enumerate(rows.iterrows(), start=1):
        docs.append(
            f"[Doc {i}] "
            f"n_reworks={r['n_reworks']}, "
            f"success_rate={r['success_rate']:.2f}\n"
            f"{r['context']}"
        )
    docs_block = "\n\n".join(docs)

    # 3. Construct the LLM prompt in English
    prompt = f"""
Below are the historical repair records and success rates for:

symptom_label = "{fail_symptom_label}"

{docs_block}

Based on the information above, please summarize:

1. The most likely root causes
2. Recommended debug / repair steps (in bullet points)
3. Any warnings or risky repair methods with low success rate

Please respond in English.
"""

    return rows, prompt


In [25]:
rows, prompt = recommend_repair_simple(
    "Checking BMC boot readiness and get BMC IP Failed"
)
print(prompt)


Below are the historical repair records and success rates for:

symptom_label = "Checking BMC boot readiness and get BMC IP Failed"

[Doc 1] n_reworks=558, success_rate=0.89
Symptom label: Checking BMC boot readiness and get BMC IP Failed | Symptom message: Failed to get IP address via telnet
Response from command 'ip addr | grep global | grep eth1':
ip addr | grep global | grep eth1
root@roux:~#  | Debug tree steps:
[Cycle 1] Category: Retest
[Cycle 1] Steps: 1. Retest on a different slot
[Cycle 2] Category: Repair 1
[Cycle 2] Steps: 1. Quick visual inspection on agora
1.1 If physical damage found, replace agora
1.2. If no physical damage found, reseat agora
[Cycle 3] Category: Repair 1
[Cycle 3] Steps: 1. Replace agora
[Cycle 4] Category: Debug 1
[Cycle 4] Steps: 1. Try new/golden agora
[Cycle 5] Category: Debug 2
[Cycle 5] Steps: 1. Confirm if motherboard replacement is needed | Historical reworks for this label: 558 cases, success rate: 0.89

[Doc 2] n_reworks=558, success_rate=0.

In [26]:
import difflib

def retrieve_by_label_and_msg(
    rag_df_sub,
    fail_symptom_label,
    fail_symptom_message=None,
    top_k=5
):
    # 1) 先用 label_norm 篩出同一個 symptom_label
    key = str(fail_symptom_label).strip().lower()
    cand = rag_df_sub[rag_df_sub["label_norm"] == key].copy()
    
    if cand.empty:
        return cand  # 找不到就直接回空的 DataFrame

    # 2) 如果有丟 symptom_message 進來，就算字串相似度
    if fail_symptom_message is not None:
        q = str(fail_symptom_message).strip().lower()
        cand["msg_sim"] = (
            cand["symptom_message"]
            .fillna("")
            .str.lower()
            .apply(lambda s: difflib.SequenceMatcher(None, s, q).ratio())
        )
    else:
        cand["msg_sim"] = 0.0  # 沒給 message 就全部 0

    # 3) 依照「訊息相似度 → rework 次數 → 成功率」排序
    cand = cand.sort_values(
        ["msg_sim", "n_reworks", "success_rate"],
        ascending=[False, False, False]
    )

    return cand.head(top_k)


In [27]:
rows = retrieve_by_label_and_msg(
    rag_df_sub,
    fail_symptom_label="Checking BMC boot readiness and get BMC IP Failed",
    fail_symptom_message="Failed to get IP address via telnet",
    top_k=3
)

rows[["symptom_label", "symptom_message", "n_reworks", "success_rate", "msg_sim"]]


,symptom_label,symptom_message,n_reworks,success_rate,msg_sim
1,Checking BMC boot readiness and get BMC IP Failed,Failed to get IP address via telnet\nResponse ...,558,0.890681,0.395480
2,Checking BMC boot readiness and get BMC IP Failed,Switch to agora console failed.,558,0.890681,0.181818


In [28]:
import difflib

def retrieve_by_label_and_msg(
    rag_df_sub,
    fail_symptom_label,
    fail_symptom_message=None,
    top_k=5
):
    # 1) filter by normalized label
    key = str(fail_symptom_label).strip().lower()
    cand = rag_df_sub[rag_df_sub["label_norm"] == key].copy()
    
    if cand.empty:
        return cand  # empty DataFrame

    # 2) message similarity (optional)
    if fail_symptom_message is not None:
        q = str(fail_symptom_message).strip().lower()
        cand["msg_sim"] = (
            cand["symptom_message"]
            .fillna("")
            .str.lower()
            .apply(lambda s: difflib.SequenceMatcher(None, s, q).ratio())
        )
    else:
        cand["msg_sim"] = 0.0

    # 3) sort by similarity, n_reworks, success_rate
    cand = cand.sort_values(
        ["msg_sim", "n_reworks", "success_rate"],
        ascending=[False, False, False]
    )

    return cand.head(top_k)


In [29]:
def recommend_repair(
    fail_symptom_label: str,
    fail_symptom_message: str | None = None,
    top_k: int = 3,
):
    """
    High-level helper:
    - retrieve top-k historical repair records for a given symptom label/message
    - build an English prompt that you can send to an LLM
    Returns:
        rows  : DataFrame with the selected records
        prompt: str, ready-to-use LLM prompt
    """
    # 1. retrieval
    rows = retrieve_by_label_and_msg(
        rag_df_sub,
        fail_symptom_label=fail_symptom_label,
        fail_symptom_message=fail_symptom_message,
        top_k=top_k,
    )

    if rows.empty:
        return rows, f"No historical repair data found for symptom_label = '{fail_symptom_label}'."

    # 2. build context block
    docs = []
    for i, (_, r) in enumerate(rows.iterrows(), start=1):
        docs.append(
            f"[Doc {i}]\n"
            f"symptom_label: {r['symptom_label']}\n"
            f"symptom_message: {r['symptom_message']}\n"
            f"message_similarity: {r.get('msg_sim', 0):.2f}\n"
            f"n_reworks: {r['n_reworks']}\n"
            f"success_rate: {r['success_rate']:.2f}\n"
            f"---\n"
            f"{r['context']}"
        )
    docs_block = "\n\n".join(docs)

    # 3. LLM prompt (English)
    prompt = f"""
You are a hardware debug assistant supporting AI server production.

Below are historical repair records and statistics for similar failures.

Current failure:
- Symptom label: {fail_symptom_label}
- Symptom message: {fail_symptom_message}

Historical records:
{docs_block}

Based on the information above, please provide:

1. The most likely root causes for the current failure.
2. Recommended debug / repair steps, in clear ordered bullet points.
3. Any warnings, risks, or low-success-rate actions that should be avoided or used only as a last resort.
4. If relevant, suggestions on when to escalate or run ORT / additional screening.

Please respond in concise technical English suitable for factory engineers.
"""

    return rows, prompt


In [30]:
print(prompt[:800])   # 開頭 800 個字
print(prompt[-800:])  # 結尾 800 個字


Below are the historical repair records and success rates for:

symptom_label = "Checking BMC boot readiness and get BMC IP Failed"

[Doc 1] n_reworks=558, success_rate=0.89
Symptom label: Checking BMC boot readiness and get BMC IP Failed | Symptom message: Failed to get IP address via telnet
Response from command 'ip addr | grep global | grep eth1':
ip addr | grep global | grep eth1
root@roux:~#  | Debug tree steps:
[Cycle 1] Category: Retest
[Cycle 1] Steps: 1. Retest on a different slot
[Cycle 2] Category: Repair 1
[Cycle 2] Steps: 1. Quick visual inspection on agora
1.1 If physical damage found, replace agora
1.2. If no physical damage found, reseat agora
[Cycle 3] Category: Repair 1
[Cycle 3] Steps: 1. Replace agora
[Cycle 4] Category: Debug 1
[Cycle 4] Steps: 1. Try new/golden agora
ical damage found on mobo, send to Repair 2 for mobo replacement.
2. If no physical damage found, reseat agora and gulp
[Cycle 3] Category: Debug 2
[Cycle 3] Steps: 1. Try golden agora, try golden gu

In [31]:
rows, prompt = recommend_repair(
    fail_symptom_label="Checking BMC boot readiness and get BMC IP Failed",
    fail_symptom_message="Failed to get IP address via telnet"
)

# 看一下被選到的歷史資料
rows[["symptom_label", "symptom_message", "msg_sim", "n_reworks", "success_rate"]]

# 把 prompt 貼去 ChatGPT / 你未來的 LLM
print(prompt)



You are a hardware debug assistant supporting AI server production.

Below are historical repair records and statistics for similar failures.

Current failure:
- Symptom label: Checking BMC boot readiness and get BMC IP Failed
- Symptom message: Failed to get IP address via telnet

Historical records:
[Doc 1]
symptom_label: Checking BMC boot readiness and get BMC IP Failed
symptom_message: Failed to get IP address via telnet
Response from command 'ip addr | grep global | grep eth1':
ip addr | grep global | grep eth1
root@roux:~# 
message_similarity: 0.40
n_reworks: 558
success_rate: 0.89
---
Symptom label: Checking BMC boot readiness and get BMC IP Failed | Symptom message: Failed to get IP address via telnet
Response from command 'ip addr | grep global | grep eth1':
ip addr | grep global | grep eth1
root@roux:~#  | Debug tree steps:
[Cycle 1] Category: Retest
[Cycle 1] Steps: 1. Retest on a different slot
[Cycle 2] Category: Repair 1
[Cycle 2] Steps: 1. Quick visual inspection on ago

### 建立Search Engine

In [32]:
rag_df_sub["search_text"] = (
    rag_df_sub["symptom_label"].fillna("") + " | " +
    rag_df_sub["symptom_message"].fillna("") + " | " +
    rag_df_sub["context"].fillna("")
)


In [34]:
import numpy as np
from sentence_transformers import SentenceTransformer

# 1) 選一個本地模型（先用輕量級的就好）
#    第一次跑會從網路下載，之後可以放到公司內部 mirror
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(model_name)

# 2) 準備要用來搜索的文字欄位
rag_df_sub["search_text"] = (
    rag_df_sub["symptom_label"].fillna("") + " | " +
    rag_df_sub["symptom_message"].fillna("") + " | " +
    rag_df_sub["context"].fillna("")
)

# 3) 對所有 document 生 embedding
corpus_texts = rag_df_sub["search_text"].tolist()
emb_matrix = embed_model.encode(
    corpus_texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,  # 之後算 cosine similarity 方便
)

# 建議：把 embedding 存起來，之後就不用每次重算
np.save("rag_emb_matrix.npy", emb_matrix)
rag_df_sub.to_pickle("rag_df_sub_with_search_text.pkl")


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\envs\embed310\lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [ ]:
import sys
print(sys.version)
print(sys.executable)


3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
